<a href="https://colab.research.google.com/github/prachijainstudent/fmml-projects-and-labs/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

In [ ]:
###The size of the validation set can indeed impact the accuracy estimate of your model, and it’s a bit of a balancing act. Here’s why:
###Increasing the Validation Set Size: If you increase the percentage of your data that is used for validation, you’ll have a larger and potentially more diverse set of examples to evaluate your model. This could lead to a more reliable and stable estimate of how well your model is likely to perform on unseen data. However, this also means that you’re reducing the amount of data available for training your model, which could negatively impact its ability to learn.
###Decreasing the Validation Set Size: Conversely, if you decrease the percentage of your data used for validation, you’ll have more data available for training. This could potentially improve your model’s performance on the training set and allow it to learn more complex patterns in the data. However, having a smaller validation set means you’re evaluating your model on fewer examples, which could lead to a less reliable estimate of its performance on unseen data.
###In practice, it’s common to use something like an 80/20 or 70/30 split for training/validation data. However, these are not hard-and-fast rules and the optimal split can depend on various factors like the total amount of data available and the specific problem you’re trying to solve.
###It’s also worth noting that techniques like cross-validation can help mitigate some of these issues by effectively using different portions of your data for training and validation across multiple iterations.

In [ ]:
###The size of the training and validation sets can significantly affect the model’s ability to predict the accuracy on the test set. Here’s how:
###Training Set Size: The size of the training set is crucial because it determines how well the model can learn. With more training data, the model has more examples to learn from, which can improve its ability to capture the underlying patterns in the data. This can lead to better performance on the test set. However, if the training set is too large, it might lead to longer training times and require more computational resources.
###Validation Set Size: The validation set is used to tune model parameters (like learning rate, number of layers in a neural network, etc.) and to provide an unbiased evaluation of the model’s performance during training. If the validation set is too small, it might not be representative of the overall data distribution, leading to an inaccurate estimate of the test accuracy. On the other hand, if it’s too large, you’re reducing the amount of data available for training.
###In practice, it’s common to use something like an 80/20 or 70/30 split for training/validation data. However, these are not hard-and-fast rules and the optimal split can depend on various factors like the total amount of data available and the specific problem you’re trying to solve.

In [ ]:
###The percentage of data you should reserve for the validation set can depend on several factors, including the total size of your dataset and the complexity of your model. However, a common practice is to use about 20% to 30% of the data for validation.
###This split often provides a good balance, allowing the model to have enough data to learn effectively (70-80% for training) while still having a substantial amount of unseen data to validate the performance and generalize well (20-30% for validation).
###But remember, these are not strict rules. Depending on the specifics of your project, you might find that a different split works better. For very large datasets, you might need less than 20% for the validation set. Conversely, for small datasets, techniques like cross-validation might be more appropriate.
###It’s always a good idea to experiment with different splits and validation techniques to see what works best for your specific problem.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


In [ ]:
### Yes, averaging the validation accuracy across multiple splits, a process known as cross-validation, can indeed provide more consistent and reliable results. This is because it reduces the variance associated with a single trial of a train/test split.
### By doing multiple splits and averaging the results, you’re effectively using more of your data for both training and testing, which can lead to a more accurate estimation of how well your model will perform on unseen data.
### However, it’s important to note that while cross-validation can provide a better estimate of performance, it doesn’t necessarily improve the performance of the model itself. It’s simply a more robust way to assess model performance.
### Also, cross-validation can be computationally expensive if you have a large dataset or if you’re using a complex model, as it requires fitting the model multiple times on different subsets of the data. So it’s always a trade-off between computational resources and getting a more reliable performance estimate.



In [ ]:
### Yes, averaging the validation accuracy across multiple splits, a process known as cross-validation, can provide a more accurate estimate of test accuracy.
###In traditional train/test split, we might just be lucky (or unlucky) with our random split of data. The test accuracy we get in this case is dependent on how the data is split into training and testing sets.
###On the other hand, in cross-validation, we perform multiple splits, train our model for each split, and average the results. This process uses more of the data for validation, and hence provides a better estimate of the test accuracy.
###However, it’s important to note that while cross-validation can provide a better estimate of performance, it doesn’t necessarily improve the performance of the model itself. It’s simply a more robust way to assess model performance.

In [ ]:
###In the context of machine learning, the number of iterations often refers to the number of times the learning algorithm will go through the training set to update the model parameters.
###Increasing the number of iterations can potentially improve the performance of the model on the training set because it gives the model more opportunities to learn from the data. However, it doesn’t necessarily mean that it will improve the estimate of test accuracy.
###Here’s why: If we iterate too many times, the model may start to overfit to the training data, meaning it becomes too specialized to the training set and performs poorly on unseen data (test set). This is why we use a separate validation set or cross-validation to tune hyperparameters like the number of iterations.
###In terms of cross-validation, increasing the number of folds (i.e., how many times we split our data) can give a better estimate of test accuracy up to a point. More folds mean more different ways we train and test on our data which can give us a better average estimate. However, there are diminishing returns after a certain point, and more folds also mean more computation time.

In [ ]:
###Increasing the number of iterations is unlikely to compensate for a very small training or validation dataset. Here’s why:
###Overfitting: If your training dataset is small, increasing the number of iterations might lead to overfitting. This means that your model might perform well on the training data but poorly on unseen data because it has essentially memorized the training data instead of learning general patterns.
###Validation Set Size: The purpose of the validation set is to provide an unbiased evaluation of the model fit on the training dataset while tuning model hyperparameters. If the validation set is too small, it might not be representative of the data distribution and might give an unreliable estimate of the model performance.
###However, there are some strategies you can use when dealing with small datasets:
###Data Augmentation: This involves creating new data based on modifications of your existing data. For example, in image data, you can use techniques like rotation, scaling, or cropping to create new data.
###Transfer Learning: This involves using a pre-trained model on a larger dataset and fine-tuning it on your smaller dataset. This allows your model to leverage the patterns learned from the larger dataset.
###Cross-Validation: As mentioned earlier, cross-validation involves splitting your dataset into multiple subsets and training your model on these subsets multiple times. This can help provide a more reliable estimate of model performance.
###Remember, each problem is unique and might require different strategies. It’s always a good idea to experiment with different approaches and see what works best for your specific problem.